In [11]:
from querymodule import fume_query, query_to_list, total_energy, total_time_sash_open
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil import tz
import requests
import json
import matplotlib.pyplot as plt
import math
import scipy.stats as st

from dateutil import parser
import matplotlib.dates as mdates

points = pd.read_csv("pointnames.csv")

In [22]:
sash_point = points["Sash"][0]
server  = points["Server"][0]
time_start = str(datetime(2022, 1,1))
time_end = str(datetime(2022, 12,31))
sash_points = points["Sash"]


# def has_been_closed(sash_point,server,start,end):
#     query = fume_query(sash_point,server,start,end)
#     inches = [i[0] for i in query]
#     ratio = inches.count(min(inches))/len(inches)
#     return(ratio)

#ratio_list = [has_been_closed(sash_points[i], server,time_start,time_end) for i in range(len(sash_points))]
#print(ratio_list)

In [23]:
def sash_mean_sd(sash,server,start,end):
    query = fume_query(sash,server,start,end)
    return [np.std(query["value"]), np.mean(query["value"])]

big_list = [sash_mean_sd(sash_points[i], server,time_start,time_end) for i in range(len(sash_points))]


<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

In [40]:
#big_list
sd_list = [i[0] for i in big_list]
mean_list = [i[1] for i in big_list]
data = {"Name":points["Lab"], "Mean": mean_list, "Stdev":sd_list}
ndf = pd.DataFrame(data)
display(ndf)

,Name,Mean,Stdev
0,g54,5.905803,3.876972
1,141,3.105352,0.921041
2,143,3.218519,1.175734
3,147b,4.020706,1.958864
4,153,2.943679,1.508009
...,...,...,...
61,445b,1.423930,1.130280
62,453,4.319038,2.259018
63,453b,3.239343,1.693759
64,457,2.450453,2.156983


In [41]:
import plotly.express as px
fig = px.bar(ndf, x = "Name", y = "Mean",title="Mean Sash Heights Across Labs")
#fig.write_image("C:\\Users\\Rob\\Desktop\\ESW\\Fume\\energy_across_labs.png",scale=4)
fig.show()

In [42]:
fig = px.bar(ndf, x = "Name", y = "Stdev",title="Standard Deviation of Sash Heights Across Labs")
#fig.write_image("C:\\Users\\Rob\\Desktop\\ESW\\Fume\\energy_across_labs.png",scale=4)
fig.show()

In [43]:
ndf["Candidate for Hibernation"] = np.where((ndf["Stdev"]<0.25)&(ndf["Mean"]<0.5), True,False)

In [44]:
ndf

,Name,Mean,Stdev,Candidate for Hibernation
0,g54,5.905803,3.876972,False
1,141,3.105352,0.921041,False
2,143,3.218519,1.175734,False
3,147b,4.020706,1.958864,False
4,153,2.943679,1.508009,False
...,...,...,...,...
61,445b,1.423930,1.130280,False
62,453,4.319038,2.259018,False
63,453b,3.239343,1.693759,False
64,457,2.450453,2.156983,False
